In [82]:
from watsonx import WatsonxAI
import warnings

warnings.filterwarnings("ignore")

from dotenv import load_dotenv
from os import getenv

load_dotenv()
BRAVE_API_KEY = getenv("BRAVE_API_KEY", None)


proxy = "proxy.us.ibm.com:8080"

wx = WatsonxAI()
wx.connect()

In [83]:
def gen_naive_answer(question):
    """
    get the AI answer the question directly.
    """

    prompt = f"""<|user|>
please answer the question.
<<SYS>>
question: `{question}`
<</SYS>>
<|assistant|>
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

# Pattern 1: Reflection

do review and revise to improve the quality of output

In [ ]:
def gen_review(draft):
    prompt = f"""<|user|>
please review the draft provided.
-check if any illogical content.
-check if any unclear content.
-give contructive suggestion.
draft: `{draft}`
<|assistant|>
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

def gen_revise(content,review):
    prompt = f"""<|user|>
please revise the content base on the review provided.
-well adopt the suggestion provided.
-generate the revised content only.
content: `{content}`
review: `{review}`
<|assistant|>
revised version:
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

In [ ]:
requirement = "plan a trip to jakarta for 3 days"

draft = gen_naive_answer(requirement)
print("draft:")
print(draft)

review = gen_review(draft)
print("review:")
print(review)

revise = gen_revise(draft,review)
print("revise:")
print(revise)

# Pattern 2: Planning

In [ ]:
def gen_plan(goal):
    """
    break down the big task to smaller sub-tasks. that easier to execute
    """
    
    prompt = f"""<|system|>
-dont overcomplicate.
-think logically.
-think step by step.
-dont guess.
-you trust the result of the tools more than your memory.
-you trust the knowledge provide more than your memory.
-you trust the status provided. no need to check on it.
-understand the goal in detail.
-understand the goal with common sense.
-understand the status provided.
-list out what knowlege you need to satisfy the goal.
-leverage tool if possible.
<|user|>
please generate a concise and solid plan to satisfy the goal provided.
-please breakdown complex task to simple tasks.
<<SYS>>
goal: `{goal}`
<</SYS>>
-DONT generate result of the steps.
-DONT generate status.
-**DONT** generate **Final Result:**
- generate steps.
<|assistant|>
steps:
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return answer

def gen_next_step(goal,plan,status): 
    prompt = f"""<|system|>
<|user|>
please check the status against the goal and plan provided.
-understand the goal provided in detail.
-understand the plan provided in detail.
-understand the status provided in detail.
-revise the plan base on the goal and status in need.
-generate next step base on status if has next step.
-print 'complete' if no next step.
<<SYS>>
goal: `{goal}`
plan: `{plan}`
status: `{status}`
output format: `step number`. `step description` [`completed`]
-mark the step if completed.
-generate next step and if complete.
<</SYS>>
<|assistant|>
steps:
"""
    
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT).replace("```","")
    return answer

In [ ]:
goal = "get all prime number with 100"

plan = gen_plan(goal)
print("plan:")
print(plan)

next_step = gen_next_step(goal,plan,"I know what prime number is")
print("1 next step:")
print(next_step)

next_step = gen_next_step(goal,plan,"got all number from 1 to 100")
print("2 next step:")
print(next_step)

next_step = gen_next_step(goal,plan,"I got the prime numbers")
print("3 next step:")
print(next_step)

plan:
1. understand the concept of prime numbers.
2. list out all numbers from 1 to 100.
3. identify which numbers are prime.
4. list out all prime numbers.
1 next step:
1. Understand the concept of prime numbers. [Completed]
2. List out all numbers from 1 to 100.
3. Identify which numbers are prime.
4. List out all prime numbers.
Next step: List out all prime numbers.
2 next step:
1. understand the concept of prime numbers. [completed]
2. list out all numbers from 1 to 100. [completed]
3. identify which numbers are prime.
4. list out all prime numbers.
next step: Identify which numbers from 1 to 100 are prime.
3 next step:
1. Understand the concept of prime numbers. [Completed]
2. List out all numbers from 1 to 100. [Completed]
3. Identify which numbers are prime. [Completed]
4. List out all prime numbers. [Completed]
Since all steps have been completed, the process is now complete.


# Pattern 3: Use Tool

use tool to employ external data or trigger external tool.

In [3]:
import inspect
import ast
import json

def get_function_signature_and_docstring(func):
    source_lines = inspect.getsourcelines(func)[0]
    source = ''.join(source_lines)

    # Parse the function definition using AST (Abstract Syntax Trees)
    parsed_func = ast.parse(source)

    # Extract function signature and docstring
    func_signature = ''
    docstring = ''
    return_type = ''
    for node in parsed_func.body:
        if isinstance(node, ast.FunctionDef):
            func_signature = source_lines[node.lineno - 1].strip()
            docstring = ast.get_docstring(node) or ''
            # Get return type annotation
            return_type = node.returns.id if node.returns else 'None'
            break

    return func_signature, docstring, return_type

def format_function_info(func):
    func_signature, docstring, return_type = get_function_signature_and_docstring(func)
    function_name = func.__name__

    # Extract parameters from a JSON schema-like structure
    parameters = {
        "type": "object",
        "properties": {}
    }

    # Get the function's parameters and their annotations
    func_params = inspect.signature(func).parameters
    for param_name, param in func_params.items():
        param_type = param.annotation
        param_description = ""  # You can add a description here if needed
        parameters["properties"][param_name] = {
            "type": str(param_type),
            "description": param_description
        }

    # Set the required parameters based on function annotations
    required_params = [param_name for param_name, param in func_params.items() if param.default == inspect.Parameter.empty]
    parameters["required"] = required_params

    return {
        "name": function_name,
        "parameters": parameters,
        "description": docstring,
        "return_type": return_type
    }

def get_tools_prompt(tools):
    prompt = "["
    # Get the source code of the function
    for tool in tools:
        info = format_function_info(tool)
        prompt = prompt + f"{info}\n"
    prompt = prompt + "]"
    return prompt

def pick_tool(job,thought,tools):
    """
    pick a tool from toolbox, that best match the need for requirement.
    """

    prompt = f"""<|system|>
You are a helpful assistant with access to the following function calls. 
- Your task is to produce a sequence of function calls necessary to generate response to the user utterance. 
- well understand the thought to articulate the task.
- well understand the description and parameters of tools.
= select appropriate parameter for the tool.
- only pick the function required and provided in tools.
- show "missing tool" if no appropriate function provided.
- DONT explain.
output format: [{{"name": "function", "arguments": {{"arg": "value"}}}}]
<<TOOLS>>
{get_tools_prompt(tools)}
<</TOOLS>>
<|user|>
thought: `{thought}`
job: `{job}`
<|assistant|>
"""
    # print(prompt)
    # print(">>"+step)
    tool = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)
    return tool

def use_tool(func_json_str):
    """
    you already hv the tool, let run it.
    """
    try:
        # Parse the JSON string into Python objects
        function_calls = json.loads(func_json_str)
        
        # Dictionary to store results
        results = {}
        
        # Iterate over each function call in the list
        for call in function_calls:
            func_name = call.get("name")
            args = call.get("arguments", {})
            print(f"calling {func_name}({args})...")
            
            # Check if the function exists in the global namespace
            if func_name in globals() and callable(globals()[func_name]):
                try:
                    # Retrieve the function object
                    func = globals()[func_name]
                    
                    # Call the function with the provided arguments
                    result = func(**args)
                    
                    # Store the result
                    results[f"{func_name}({args})"] = result
                except Exception as e:
                    # Handle exceptions raised by the function
                    results[f"{func_name}({args})"] = f"Error executing {func_name}: {str(e)}"
            else:
                # Function not found or not callable
                results[f"{func_name}({args})"] = f"Function '{func_name}' not found or is not callable."
        
        return results
    
    except json.JSONDecodeError as jde:
        return {"JSONDecodeError": f"Invalid JSON input: {str(jde)}"}
    except Exception as ex:
        return {"Error": f"An unexpected error occurred: {str(ex)}"}
    
def gen_summary_from_knowledge(question,knowledge):
    prompt = f"""<|user|>
help answer the question provided base on the knowledge provided.
-focus on what you found instead of not found.
-if no answer provide, just say you dont know.
<<SYS>>
question = `{question}`
knowledge = `{knowledge}`
<</SYS>>
<|assistant|>
"""
    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT,8000)
    return answer

In [4]:
import requests
import wikipediaapi
import requests
import yfinance as yf
import pytz
from datetime import datetime

from dotenv import load_dotenv
from os import getenv

load_dotenv()
PROXY = getenv("PROXY", None)

def query_wikipedia(topic)->str:
    """
    only use it for ONE keyword/topic. 
    NEVER applicable to long query.
    get detail definition and information of particular topic
    only ONE keyword/topic will be support. please use appropriate one.
    only support very specific short keyword or topic.

    Parameters:
    topic (str): The ONE keyword/topic for which the information is required.

    Returns:
    str: A dictionary containing the title, summary, and URL of the Wikipedia page for the given topic. If the page is not found, it returns "Page not found."
    """
    wiki = wikipediaapi.Wikipedia('watsonx',proxies={'http': PROXY,'https':PROXY})
    page = wiki.page(topic)

    if page.exists():
        return {
            "title": page.title,
            "summary": page.summary,
            "url": page.fullurl
        }
    else:
        return "Page not found."

def get_stock_data(ticker, period='5d', interval='1d')->str:
    """
    get stock price. it is for stock data only.
    the input is ticker (should be a stock code)
    dont put other thing to ticker.
    """
    session = requests.Session()
    session.proxies = {
        'http': PROXY,
        'https': PROXY
    }

    stock = yf.Ticker(ticker, session=session)
    data = stock.history(period=period, interval=interval)
    return data

def get_current_time(location)->str:
    """
    Get the current time by location.

    Args:
        location (str) need to be in format of 'Europe/London'

    Returns:
        str: The current time in the specified location.

    Example:
        get_current_time('Europe/London')
    """
    try:
        location_time = datetime.now(pytz.timezone(location))
        return location_time.strftime("%Y-%m-%d %H:%M:%S")
    except Exception as e:
        return f"Error: {str(e)}"

tools = [query_wikipedia,get_stock_data,get_current_time]

In [5]:
def run_agentic_from_tool(question,tools,debug=False):
    # print(get_tools_prompt(tools))
    # thought = gen_thought(question)
    action = pick_tool(question,"",tools)
    if debug:
        print(f"action:{action}")
    answer = use_tool(action)
    if debug:
        print(f"tool result:{answer}")
    summary = gen_summary_from_knowledge(question,answer)
    return summary

In [7]:
answer = gen_naive_answer("how deep is Mariana Trench")
print("naive:")
print(answer)

print("use tool:")
answer = run_agentic_from_tool("how deep is Mariana Trench",tools)
print(answer)

naive:
The Mariana Trench, located in the western Pacific Ocean, is approximately 36,070 feet (10,994 meters) deep at its deepest point, known as the Challenger Deep. This makes it the deepest part of the world's oceans.
use tool:
calling query_wikipedia({'topic': 'Mariana Trench'})...
The Mariana Trench, located in the western Pacific Ocean, is the deepest oceanic trench on Earth. Its maximum known depth is 10,984 ± 25 meters (36,037 ± 82 feet; 6,006 ± 14 fathoms; 6.825 ± 0.016 miles). This depth is more than 2 kilometers (1.2 miles) farther from sea level than the peak of Mount Everest. The pressure at the bottom of the trench is 1,086 bar (15,750 psi), and the temperature ranges from 1 to 4 degrees Celsius (34 to 39 degrees Fahrenheit). One-celled organisms called monothalamea have been found at a record depth of 10.6 kilometers (35,000 feet; 6.6 miles) below the sea surface.


In [6]:
answer = gen_naive_answer("analysis the latest stock price of IBM")
print("naive:")
print(answer)

print("use tool:")
answer = run_agentic_from_tool("analysis the latest stock price of IBM",tools)
print(answer)

naive:
As of my last update, I don't have real-time data or the ability to browse the internet. However, I can guide you on how to find the latest stock price of IBM. You can check financial websites like Yahoo Finance, Google Finance, or Bloomberg for the most recent stock prices. Please ensure you're using a reliable source and that the information is up-to-date.
use tool:
calling get_stock_data({'ticker': 'IBM'})...
Based on the provided knowledge, the latest stock price of IBM as of November 18, 2024, was $208.089996. This is the closing price for that day. The volume of shares traded on that day was 2658508. There were no dividends or stock splits reported for this period.


In [8]:
answer = gen_naive_answer("when it is now in Sydney")
print("naive:")
print(answer)

print("use tool:")
answer = run_agentic_from_tool("when it is now in Sydney",tools)
print(answer)

naive:
The current time in Sydney, Australia is [insert current time here]. Please note that Sydney observes Daylight Saving Time from early October to early April, during which the time zone is UTC+10:30. The rest of the year, it follows UTC+11:00.
use tool:
calling get_current_time({'location': 'Australia/Sydney'})...
The current time in Sydney is 13:47:09 on November 19, 2024.


# Pattern 4: Multi Agent

adopt different personale to contribute to the task in different perspective

In [25]:
class Agent:
    role = ""
    system_prompt = ""
    goal = ""
    next_agent = None

def gen_multigent(task,agent,turns=5,debug=False):
    memory = []
    current_turn = 1
    current_agent = agent
    while True:
        if current_turn >= turns:
            break
        print(f"turn # {current_turn} - role: {current_agent.role}")
        answer = gen_role_input(task,
                                current_agent,
                                memory)
        if debug:
            print(answer)
        memory.append(answer)
        if current_agent.next_agent is None:
            break
        current_agent = current_agent.next_agent
        current_turn += 1

    prompt = f"""<|user|>
summarize the knowledge provided against the task provided.
-make good readability and structure.
-dont miss any argument made.
-consolidate similar points.
task: `{task}`
knowledge: `{memory}`
<|assistant|>"""

    answer = wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT)

    return answer

def gen_role_input(task,agent,memory):
    prompt = f"""<|system|>
you are {agent.role} in the conversation provided.
well aware of your goal.
well aware of the task of discussion.
{agent.system_prompt}
<|user|>
goal: `{agent.goal}`
task: `{task}`
conversation: `{memory}`
output:
-generate the output directly.
-dont explain.
<|assistant|>"""
    answer = {
        "role": agent.role,
        "content": wx.watsonx_gen(prompt,wx.GRANITE_3_8B_INSTRUCT,max_output=1000)
    }
    return answer

In [27]:
answer = gen_naive_answer("discuss on the topc `should we pay higher tax`")
print(answer)

The topic of whether or not we should pay higher taxes is a complex one, with valid arguments on both sides. Here are some points to consider:

1. **Public Services and Infrastructure**: Higher taxes can fund public services and infrastructure that benefit society as a whole, such as education, healthcare, and transportation systems.

2. **Economic Inequality**: Critics argue that higher taxes disproportionately affect lower-income individuals, exacerbating economic inequality.

3. **Government Efficiency**: Some people believe that higher taxes could lead to more efficient government spending and reduced waste.

4. **Economic Growth**: Others argue that high tax rates can discourage investment and entrepreneurship, potentially slowing economic growth.

5. **Social Safety Nets**: Higher taxes can help fund social safety nets, providing support for those in need.

6. **Fiscal Responsibility**: Some view higher taxes as a way to ensure fiscal responsibility and prevent excessive governme

In [26]:
pros = Agent()
cons = Agent()

pros.role = "supporter"
pros.system_prompt = "give positive argument for the discussion, rebuttle on argument objective to yours."
pros.goal = "provide convincing supportive argument on the topic in discussion."
pros.next_agent = cons

cons.role = "critic"
cons.system_prompt = "give negative argument for the discussion, rebuttle on argument objective to yours."
cons.goal = "provide convincing critic argument on the topic in discussion."
cons.next_agent = pros

task = "discuss on the topc `should we pay higher tax`"

answer = gen_multigent(task,pros)
print(answer)

turn # 1 - role: supporter
turn # 2 - role: critic
turn # 3 - role: supporter
turn # 4 - role: critic

The task is to discuss the topic "Should we pay higher taxes?" and provide a summary of the arguments presented by both supporters and critics.

Supporters argue that higher taxes can lead to improved public services such as education, healthcare, and infrastructure, which can benefit society as a whole. They also believe that higher taxes can reduce income inequality, promote environmental sustainability, and provide necessary funds for social safety nets. Supporters argue that these benefits outweigh the potential drawbacks of higher taxes, such as economic burden and disincentive for work and investment.

Critics, on the other hand, present several counterarguments against higher taxes. They argue that higher taxes can place a significant financial burden on individuals and businesses, potentially stifling economic growth and investment. Critics also believe that higher taxes may d